# Classifying Embeddings

## Introduction

**Learning Objectives**

- Load in and prepare the embeddings
- Design a MLP classifier
- Train the classifier

Now that we have gained **embeddings** of our *wt-mt* AA pair we can utilise it as an input to a neural network. If you remember back to the first section, each mutation was accompanied with a label which indicated whether the missense mutation is **benign (0)** or **pathogenic (1)**.

Our aim is to design and train a neural net that can accurately predict this given an embedding.

## Loading the Data

To train our model we will need the data from the train *pickle* we saved in the previous section. Pickles can be loaded in much the same way they are saved, through a file handling `with` statement.

In [ ]:
import pickle
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

NOTE BOX

Due to the file upload contraints with GitHub only files **below 100 MB** can be uploaded. We have therefore split up the embedding pickles into chunks that can be read in through a `for-loop`. See the code for this in the collapsable cell below.

In [ ]:
import glob

# Load all chunks
chunks = []
for chunk_file in sorted(glob.glob("./data/embeddings_train_chunk_*.pkl")):
    with open(chunk_file, 'rb') as file:
        chunks.append(pickle.load(file))

# Combine all chunks
test_dict = {
    'target_id': [],
    'embeddings': [],
    'labels': []
}

for chunk in chunks:
    test_dict['target_id'].extend(chunk['target_id'])
    test_dict['embeddings'].extend(chunk['embeddings'])
    test_dict['labels'].extend(chunk['labels'])

X = np.array(test_dict['embeddings'])
y = test_dict['labels']

In [ ]:
with open("./data/embs_for_train.pkl", 'rb') as file:
    train_dict = pickle.load(file)

X = np.array(train_dict['embeddings'])
y = train_dict['labels']

For training we only need the **embeddings** and **labels**, the IDs can be left behind.

### Split data into training and test sets 

Previously when working with our machine learning models, we have just been using **train** and **test**. Here, we will use a further set, the **validation** set, a subset of the training dataset.

Each subset of the data serves a unique purpose:

| Split | Purpose | Proportion of Total Data | Usage Frequency |
| ----- | ------- | ------------------------ | --------------- |
| **Training set** | Used to train the model (similar to the course materials you study during the semester). | ~60-80% | Always |
| **Validation set** | Used to fine-tune the model (akin to taking practice exams before the final exam). | ~10-20% | Often, but not always |
| **Testing set** | Used to evaluate the model’s performance, providing an indication of how well it has learned (comparable to the final exam at the end of the semester). | ~10-20% | Always |

NOTE BOX

In real-world scenarios, this step is typically performed at the very beginning of a project. The test set must remain isolated from the training and validation data to ensure the model's performance reflects its ability to **generalise** to unseen examples.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                            test_size=0.2,
                                                            shuffle=True,
                                                            stratify=y,
                                                            random_state=42)
print('X_train shape: ', X_train.shape)
print('X_valid shape: ', X_valid.shape)

We will also be needing our validation dataset for the next section, **evaluation**, so these need to be saved.

In [ ]:
with open("./data/validation_data.pkl", 'wb') as file:
    val_dict = {'embeddings': X_valid, 'labels': y_valid}
    pickle.dump(val_dict, file)
    print('Data saved successfully')

## Batch Loader

As in previous lessons we will require the `DataLoader` and `Dataset` class from *PyTorch*. However, previously we were working with datasets built in the *PyTorch* system, whereas now we are working with a custom datset.

To have our data work with `DataLoader` we will need to create a custom **dataset** class, subclassed from `Dataset`.

### Dataset

`Dataset` has three most commonly used functions: `__init__`, `__len__`, and `__getitem__`.:

`__init__`: This function is the initializer or constructor for the Dataset class. It's where you define and initialize the dataset's properties, such as file paths, data lists, transformations, or any other initialization parameters needed for your dataset. Essentially, this method sets up the dataset object when it is first created, preparing it for use.

`__len__`: This method returns the total number of items or samples in the dataset. When this function is called, it should provide the length of the dataset, which is typically the total number of samples that are available to be returned. This is useful for understanding the size of the dataset and is used by DataLoader to determine how many batches are needed to iterate over the entire dataset.

`__getitem__`: This method is responsible for fetching and returning a single item from the dataset. Given an index, `__getitem__` retrieves the specific sample at that index. This method is essential for indexing and iterating through the dataset. It can include the logic for data loading, preprocessing, and transformation, returning the data sample (and possibly its label) in the format required for model training.

In [ ]:
class EmbDataset(Dataset): # Subclass Dataset
    def __init__(self, X, y):
        super().__init__()

        self.seq = torch.tensor(X)
        self.label = torch.tensor(y)

    def __len__(self):
        return len(self.seq)

    def __getitem__(self, index):
        return self.seq[index], self.label[index]

In [ ]:
BATCH_SIZE = 32

train_dataset = EmbDataset(X_train, y_train)
val_dataset = EmbDataset(X_valid, y_valid)

# To dataloader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True) 
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

## Model set up

Going back to **AI1** we will be designing a **multi layer perceptron** (MLP).

We will be introducing a new **activation layer**, a variation of the **ReLU**. function. Where the **ReLU** activation function zeros all output below a threshold, the [Leaky ReLU](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.LeakyReLU) allows a small gradient when the unit is not active. This can help keep the gradient from vanishing.

A vanishing gradient can make it difficult for the network to learn, as the weights in the network become very small and the learning process slows down or stops.

In [ ]:
class MLP_LeakyReLu_binary(nn.Module):
    """Simple MLP Model for Classification Tasks."""

    def __init__(self, num_input, num_hidden, num_output=1):
        """
        Initialise the MLP Classifier with input, hidden, and output layers.

        Args:
            num_input (int): The number of input features.
            num_hidden (int): The number of units in the hidden layer.
            num_output (int): The number of output classes.
        """
        # Call the parent class (nn.Module) initialiser
        super().__init__()

        # Define the first linear layer (input to hidden)
        self.hidden = nn.Linear(num_input, num_hidden)
        
        # Define the second part of the network (hidden to output)
        # Includes Dropout for regularisation, LeakyReLU activation, and a final linear layer
        self.predict = nn.Sequential(
            nn.Dropout(0.5),            # Apply dropout with 50% probability to prevent overfitting
            nn.LeakyReLU(inplace=True), # Apply LeakyReLU activation with in-place modification
            nn.Linear(num_hidden, num_output) # Fully connected layer for output predictions
        )
        self.sigmoid = nn.Sigmoid()  # Apply sigmoid activation to the output


    def forward(self, x):
        """
        Define the forward calculation process to train the network.

        Args:
            x (torch.Tensor): Input data tensor of shape (batch_size, num_input).

        Returns:
            torch.Tensor: Output predictions of shape (batch_size, num_output).
        """
        x = self.hidden(x)  # Pass the input through the first linear layer (input to hidden)
        x = self.predict(x) # Pass the hidden layer output through the prediction layers
        x = self.sigmoid(x) # Apply sigmoid activation to the output

        return x            # Return the final output

As before, lets check for any bugs by giving it some randomly generated data of the right shape.

In [ ]:
n_input = X_train[0].shape[0]

tmp_input = torch.randn(32, n_input)

# Initialise a model object
model = MLP_LeakyReLu_binary(n_input, 1280, 1)
output = model(tmp_input)

print(output[:5])

NOTE BOX

Even though the problem is a **binary** (2 class) problem, the output can be set to **one**. This is because the two results are dependent on one another, i.e. if it is positive it can not be negative , and vice versa. We can therefore increase efficency for the training and prediction by reducing the final neuron to one, reducing the parameters and the output tensor.

Checking out the trainable parameters, it is a modest **3.2 million**.

In [ ]:
# The trainable parameters
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

## Training setup

The training setup builds upon what we have seen in the previous lessons.

Here is a breakdown of the training setup changes:

- **loss function**: Binary cross-entropy. Previously we have used cross-entropy for classification tasks as we had a multi-class classification problem. Here we have a binary classification problem, so we can use binary cross-entropy.
- **optimiser**: The Adam optimiser. Previously we have used the SGD optimiser.

In [ ]:
def trainer(train_loader, val_loader, model, config):
    """
    Train a PyTorch model with early stopping and validation.

    Args:
        train_loader (DataLoader): DataLoader for the training dataset.
        val_loader (DataLoader): DataLoader for the validation dataset.
        model (nn.Module): The PyTorch model to train.
        config: Configuration object containing training parameters.
    """
    # Load training configuration parameters.
    device = config.device  # Device to use (e.g., CPU or GPU).
    early_stop = config.early_stop_patience  # Early stopping patience.
    n_epochs = config.n_epochs  # Number of training epochs.

    # Binary Cross Entropy with sum reduction.
    criterion = nn.BCELoss(reduction='sum') # sum rather than mean as values are very small

    # Adam optimiser with specified learning rate).
    optimiser = torch.optim.Adam(
        model.parameters(), lr=config.learning_rate, weight_decay=0
    )

    # Define the learning rate scheduler; reduces LR when validation loss plateaus
    lr_scheduler = ReduceLROnPlateau(optimiser, mode='min', 
                    factor=config.lr_factor, patience=config.learning_rate_patience, 
                    )

    # Initialise variables for training.
    n_epochs, best_loss, early_stop_count = n_epochs, float('inf') , early_stop

    # Training loop for each epoch.
    for epoch in range(n_epochs):

        model.train()  # Set the model to training mode.
        loss_record = []  # List to record training losses for the epoch.

        for batch in train_loader:

            optimiser.zero_grad()  # Clear gradients from the previous step.

            # Move data to the specified device.
            b_seq, b_labels = tuple(t.to(device) for t in batch)

            # Forward pass: compute predictions from the model.
            pred = model(b_seq.float())

            # Convert labels to float for BCELoss compatibility.
            b_labels = b_labels.float()

            # Calculate the loss between predictions and true labels.
            # Even though there is only one prediction we will need to index it
            loss = criterion(pred[:, 0], b_labels)

            # Backward pass: compute gradients.
            loss.backward()

            # Update model parameters
            optimiser.step()

            loss_record.append(loss.detach().item())  # Record the loss for this batch.

        # Compute the mean training loss for the epoch.
        mean_train_loss = sum(loss_record) / len(loss_record)

        # Start evaluation with validation data

        model.eval()  # Set the model to evaluation mode.
        loss_record = []  # Reset loss record for validation.

        for batch in val_loader:
            # Move data to the specified device.
            b_seq, b_labels = tuple(t.to(device) for t in batch)

            # Forward pass without gradient computation (inference mode).
            with torch.no_grad():
                b_labels = b_labels.float()  # Convert labels to float for compatibility.
                pred = model(b_seq.float())  # Generate predictions.
                loss = criterion(pred[:, 0], b_labels)  # Calculate the validation loss.

            loss_record.append(loss.item())  # Record validation loss for this batch.

        # Compute the mean validation loss for the epoch.
        mean_valid_loss = sum(loss_record) / len(loss_record)

        if epoch % 5 == 0:
            print('\n=========================== Evaluating=========================')
            # Print training and validation loss for the epoch.
            print(f'Epoch [{epoch + 1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')

        # Step the learning rate scheduler based on validation loss
        lr_scheduler.step(mean_valid_loss)

        # Check if the validation loss is the best so far.
        if mean_valid_loss < best_loss:

            best_loss = mean_valid_loss  # Update the best loss.

            # Save the current best model's state.
            torch.save(model.state_dict().copy(), './data/embedding_model.pt')

            print('\nSaving model with loss {:.3f}...'.format(best_loss))

            early_stop_count = 0  # Reset early stopping counter.
        else:
            early_stop_count += 1  # Increment the early stopping counter.

        # Stop training if the model hasn't improved for a specified number of epochs.
        if early_stop_count >= early_stop:
            print(f'Early stopping after {early_stop} epochs without improvement.')
            break # Exit the training loop.

In [ ]:
SEED = 42
torch.manual_seed(SEED)
# Set the seed for all devices (both CPU and CUDA)
torch.cuda.manual_seed_all(SEED)

# Check if GPU is available
if torch.cuda.is_available():
    # if there are multiple GPUs, choose the first one
    device = torch.device(f"cuda:0")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f'Device name: {torch.cuda.get_device_name(0)}')
else:
    print("No GPU detected! Falling back to CPU")
    # If the GPU is not available, use the CPU
    device = torch.device("cpu")

You may have noticed that the `trainer` function takes a **config** as an argumment. We can use this to pass a class with our training hyperparameters. This can be a neater way of storing changable variables in one place.

In [ ]:
class Config:
    """
    Configuration class for model training settings.
    """
    # Specify the device to be used for computation (CPU or GPU).
    device = device

    # Early stopping patience: number of epochs to wait for improvement (val loss)
    early_stop_patience = 5

    # Total number of epochs to train the model.
    n_epochs = 50

    # Learning rate for the optimiser.
    # Smaller values lead to more gradual updates, while larger values result in faster updates.
    learning_rate = 1e-4

    # Learning rate scheduler configuration
    learning_rate_patience = 3  # Number of epochs to wait before reducing learning rate
    lr_factor = 0.1 # Factor by which to reduce the learning rate

## Running Training

With our function setup all we need to do is ensure our device is set, call the **config** class, and then the trainer. The best performing models paramters will be saved to the `data` folder under the name `embedding_model.ckpt`.

In [ ]:
# Initialise the config class
config = Config()

# Call the trainer function
trainer(train_loader, val_loader, model, config)

## Summary

Our model is now trained and we can use it to classify new embeddings.

In the next section, we’ll walkthrough  demonstrate how to use this trained model to make predictions on the test set. We’ll also analyze the results to assess whether the model’s performance is consistent and reliable. Stay tuned to evaluate the true robustness of your model!